In [1]:
!pip install ibm_watsonx_ai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 4.8 MB/s eta 0:00:00
  Created wheel for ibm-cos-sdk: filename=ibm_cos_sdk-2.14.3-py3-none-any.whl size=77232 sha256=015314716e4aeb46b18ee6a4f2c0c7aeca60bf09e187096416aca93b57f0b21f
  Stored in directory: /root/.cache/pip/wheels/56/fa/85/9a1004ed234750540a7a90f34000bc1208e723f3613eaafc2b
  Created wheel for ibm-cos-sdk-core: filename=ibm_cos_sdk_core-2.14.3-py3-none-any.whl size=662101 sha256=69869410176b7f1667183ff48beaea8eec5d51bd385f1dd002fced4d4e2032f7
  Stored in directory: /root/.cache/pip/whe

In [11]:
import os
from ibm_watsonx_ai import APIClient, Credentials
import getpass
credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key=getpass.getpass("Please enter your api key (hit enter): ")
)
#IWjtMQpCrJT_SE2MJPFBh99w4v7b_q33Athk5FTi4oe3

Please enter your api key (hit enter): ··········


In [15]:
client = APIClient(credentials)

In [16]:
space_id = "a6bb01cd-7c90-4b47-b413-aa8d00d3c949"
client.set.default_space(space_id)


'SUCCESS'

In [17]:
source_project_id = "38879a89-7193-4db0-bc96-a40b01283ee4"


In [18]:
params = {
    "space_id": space_id,
}

def gen_ai_service(context, params = params, **custom):
    # import dependencies
    from langchain_ibm import ChatWatsonx
    from ibm_watsonx_ai import APIClient
    from ibm_watsonx_ai.foundation_models.utils import Tool, Toolkit
    from langchain_core.messages import AIMessage, HumanMessage
    from langgraph.checkpoint.memory import MemorySaver
    from langgraph.prebuilt import create_react_agent
    import json
    import requests

    model = "ibm/granite-3-3-8b-instruct"

    service_url = "https://us-south.ml.cloud.ibm.com"
    # Get credentials token
    credentials = {
        "url": service_url,
        "token": context.generate_token()
    }

    # Setup client
    client = APIClient(credentials)
    space_id = params.get("space_id")
    client.set.default_space(space_id)


    def decrypt_tool_secrets(secrets):
        url = "https://api.dataplatform.cloud.ibm.com"
        headers = {
            "Content-Type": "application/json",
            "Authorization": f'Bearer {context.generate_token()}'
        }

        body = {
            "secrets": secrets,
            "space_id": space_id
        }

        response = requests.post(f'{url}/wx/v1-beta/utility_agent_tools/secret/decrypt', headers=headers, json=body)

        return response.json().get("secrets")

    encrypted_secrets = [
        "gcm-agent-tools-qHi31me0EfjVZVuGAnau05GBdpyvCVyV:22Ey/IVF7iuVwtXqQFeNZg==;onvmpQr2LOx6ReFy4FIM3w==:x5yeovyHOtSaMFUWg9/OuaYqD1XGT92NC05529T0LmSfs7lr2pReI62RI4Q="
    ]
    decrypted_secrets = decrypt_tool_secrets(encrypted_secrets)

    TavilySearch_apiKey = decrypted_secrets[0]


    def create_chat_model(watsonx_client):
        parameters = {
            "frequency_penalty": 0,
            "max_tokens": 2000,
            "presence_penalty": 0,
            "temperature": 0,
            "top_p": 1
        }

        chat_model = ChatWatsonx(
            model_id=model,
            url=service_url,
            space_id=space_id,
            params=parameters,
            watsonx_client=watsonx_client,
        )
        return chat_model


    def create_utility_agent_tool(tool_name, params, api_client, **kwargs):
        from langchain_core.tools import StructuredTool
        utility_agent_tool = Toolkit(
            api_client=api_client
        ).get_tool(tool_name)

        tool_description = utility_agent_tool.get("description")

        if (kwargs.get("tool_description")):
            tool_description = kwargs.get("tool_description")
        elif (utility_agent_tool.get("agent_description")):
            tool_description = utility_agent_tool.get("agent_description")

        tool_schema = utility_agent_tool.get("input_schema")
        if (tool_schema == None):
            tool_schema = {
                "type": "object",
                "additionalProperties": False,
                "$schema": "http://json-schema.org/draft-07/schema#",
                "properties": {
                    "input": {
                        "description": "input for the tool",
                        "type": "string"
                    }
                }
            }

        def run_tool(**tool_input):
            query = tool_input
            if (utility_agent_tool.get("input_schema") == None):
                query = tool_input.get("input")

            results = utility_agent_tool.run(
                input=query,
                config=params
            )

            return results.get("output")

        return StructuredTool(
            name=tool_name,
            description = tool_description,
            func=run_tool,
            args_schema=tool_schema
        )


    def create_custom_tool(tool_name, tool_description, tool_code, tool_schema, tool_params):
        from langchain_core.tools import StructuredTool
        import ast

        def call_tool(**kwargs):
            tree = ast.parse(tool_code, mode="exec")
            custom_tool_functions = [ x for x in tree.body if isinstance(x, ast.FunctionDef) ]
            function_name = custom_tool_functions[0].name
            compiled_code = compile(tree, 'custom_tool', 'exec')
            namespace = tool_params if tool_params else {}
            exec(compiled_code, namespace)
            return namespace[function_name](**kwargs)

        tool = StructuredTool(
            name=tool_name,
            description = tool_description,
            func=call_tool,
            args_schema=tool_schema
        )
        return tool

    def create_custom_tools():
        custom_tools = []


    def create_tools(inner_client, context):
        tools = []

        config = None
        tools.append(create_utility_agent_tool("GoogleSearch", config, inner_client))
        config = {
        }
        tools.append(create_utility_agent_tool("DuckDuckGo", config, inner_client))
        config = {
            "maxResults": 5
        }
        tools.append(create_utility_agent_tool("Wikipedia", config, inner_client))
        config = {
            "maxResults": 10,
            "apiKey": TavilySearch_apiKey
        }
        tools.append(create_utility_agent_tool("TavilySearch", config, inner_client))
        config = {
        }
        tools.append(create_utility_agent_tool("WebCrawler", config, inner_client))
        return tools

    def create_agent(model, tools, messages):
        memory = MemorySaver()
        instructions = """# Notes
- When a tool is required to answer the user's query, respond only with <|tool_call|> followed by a JSON list of tools used.
- If a tool does not exist in the provided list of tools, notify the user that you do not have the ability to fulfill the request.
You are a cognitive AI-powered assistant built on IBM’s Watsonx platform, designed to streamline research and data analysis. It leverages advanced natural language processing and machine learning to understand complex queries, gather relevant information from diverse sources, and present insights in an easy-to-understand format. The agent can summarize large volumes of data, detect trends, suggest research directions, and integrate with enterprise datasets or APIs. Ideal for academic, scientific, and market research, it helps reduce time spent on manual searching while improving the accuracy and depth of findings.
Phase 1: Query Analysis and Planning
1. Receive and Deconstruct the User Query

Receive: The Orchestration Agent receives the user's initial request.

Deconstruct: Analyze the query to identify its core components:

Subject: What is the central topic? (e.g., \"sustainable energy,\" \"AI in medicine\").

Scope: What are the boundaries of the research? (e.g., \"latest research,\" \"economic impact,\" \"technological challenges\").

Output Format: What does the user expect? (e.g., \"summary,\" \"report,\" \"list of papers\").

2. Formulate a Strategic Research Plan

Generate a Plan: Based on the deconstructed query, create a step-by-step plan using the Planner Agent. This plan must be a logical sequence of actions to fulfill the request.

Example Plan:

Step 1: Conduct a broad search for recent academic papers and articles on the subject.

Step 2: Filter the search results based on relevance, publication date (e.g., last 2-3 years), and source credibility.

Step 3: For each highly relevant source, extract key findings, methodologies, and conclusions.

Step 4: Synthesize the extracted information, identifying common themes and contrasting viewpoints.

Step 5: Structure the findings into a clear, concise, and well-organized final report.

Phase 2: Information Gathering and Retrieval
1. Execute the Search Action (Research Assistant Agent)

Command: Use the generated plan to trigger search actions.

Tools:

Web Search: Use a live search tool to find the most current and publicly available information.

Knowledge Base Search: Query the internal vector database (part of the RAG system) for relevant, pre-indexed documents, research papers, and reports.

Strategy: Prioritize searches that are specific and focused to avoid irrelevant results. Use multiple search queries if the initial results are insufficient.

2. Data Extraction and Filtering

Extraction: For each search result, extract the title, author, publication date, abstract, key findings, and conclusion. Use the Summarizer Agent to quickly identify the main points of each source.

Filtering: Apply the filtering criteria from the plan. Discard any sources that are outdated, from unreliable outlets, or are not directly relevant to the user's scope.

Verification: Cross-reference information where possible. If a major finding is mentioned in one paper, see if it is supported or challenged by others.

Phase 3: Synthesis and Validation
1. Consolidate Information (Summarizer Agent)

Synthesize: Group the extracted key findings by topic or theme. For example, if the topic is \"AI in medicine,\" create sections for \"diagnostic applications,\" \"drug discovery,\" and \"ethical considerations.\"

Summarize: Create a concise summary for each major theme. These summaries should capture the essence of the findings without getting lost in technical details, and they should be grounded in the extracted data from the knowledge base.

2. Critical Review (Critic/Reflection Agent)

Evaluate: Review the synthesized summaries and the extracted data.

Check for:

Accuracy: Are the findings correctly represented?

Completeness: Are there any significant gaps in the information that need to be filled?

Relevance: Does the information directly address the user's original query?

Logical Flow: Does the synthesized information make sense as a cohesive whole?

Revise: If the review finds any deficiencies, send a command back to the Orchestration Agent to re-execute specific steps of the plan (e.g., \"Conduct a more specific search for ethical challenges of AI in medicine\").

Phase 4: Final Report Generation
1. Structure the Output (Report Generator Agent)

Format: Assemble the validated and synthesized information into a final report. The report should have a clear structure:

Introduction: Briefly state the purpose of the report based on the user's query.

Main Body: Present the findings, organized by the themes identified in the synthesis phase. Use clear headings and subheadings.

Conclusion: Summarize the key takeaways and provide a final synthesis of the research.

2. Final Review and Delivery

Final Check: Do a final review of the report for clarity, grammar, and overall quality.

Deliver: Send the final report to the user through the designated output channel."""
        for message in messages:
            if message["role"] == "system":
                instructions += message["content"]
        graph = create_react_agent(model, tools=tools, checkpointer=memory, state_modifier=instructions)
        return graph

    def convert_messages(messages):
        converted_messages = []
        for message in messages:
            if (message["role"] == "user"):
                converted_messages.append(HumanMessage(content=message["content"]))
            elif (message["role"] == "assistant"):
                converted_messages.append(AIMessage(content=message["content"]))
        return converted_messages

    def generate(context):
        payload = context.get_json()
        messages = payload.get("messages")
        inner_credentials = {
            "url": service_url,
            "token": context.get_token()
        }

        inner_client = APIClient(inner_credentials)
        model = create_chat_model(inner_client)
        tools = create_tools(inner_client, context)
        agent = create_agent(model, tools, messages)

        generated_response = agent.invoke(
            { "messages": convert_messages(messages) },
            { "configurable": { "thread_id": "42" } }
        )

        last_message = generated_response["messages"][-1]
        generated_response = last_message.content

        execute_response = {
            "headers": {
                "Content-Type": "application/json"
            },
            "body": {
                "choices": [{
                    "index": 0,
                    "message": {
                       "role": "assistant",
                       "content": generated_response
                    }
                }]
            }
        }

        return execute_response

    def generate_stream(context):
        print("Generate stream", flush=True)
        payload = context.get_json()
        headers = context.get_headers()
        is_assistant = headers.get("X-Ai-Interface") == "assistant"
        messages = payload.get("messages")
        inner_credentials = {
            "url": service_url,
            "token": context.get_token()
        }
        inner_client = APIClient(inner_credentials)
        model = create_chat_model(inner_client)
        tools = create_tools(inner_client, context)
        agent = create_agent(model, tools, messages)

        response_stream = agent.stream(
            { "messages": messages },
            { "configurable": { "thread_id": "42" } },
            stream_mode=["updates", "messages"]
        )

        for chunk in response_stream:
            chunk_type = chunk[0]
            finish_reason = ""
            usage = None
            if (chunk_type == "messages"):
                message_object = chunk[1][0]
                if (message_object.type == "AIMessageChunk" and message_object.content != ""):
                    message = {
                        "role": "assistant",
                        "content": message_object.content
                    }
                else:
                    continue
            elif (chunk_type == "updates"):
                update = chunk[1]
                if ("agent" in update):
                    agent = update["agent"]
                    agent_result = agent["messages"][0]
                    if (agent_result.additional_kwargs):
                        kwargs = agent["messages"][0].additional_kwargs
                        tool_call = kwargs["tool_calls"][0]
                        if (is_assistant):
                            message = {
                                "role": "assistant",
                                "step_details": {
                                    "type": "tool_calls",
                                    "tool_calls": [
                                        {
                                            "id": tool_call["id"],
                                            "name": tool_call["function"]["name"],
                                            "args": tool_call["function"]["arguments"]
                                        }
                                    ]
                                }
                            }
                        else:
                            message = {
                                "role": "assistant",
                                "tool_calls": [
                                    {
                                        "id": tool_call["id"],
                                        "type": "function",
                                        "function": {
                                            "name": tool_call["function"]["name"],
                                            "arguments": tool_call["function"]["arguments"]
                                        }
                                    }
                                ]
                            }
                    elif (agent_result.response_metadata):
                        # Final update
                        message = {
                            "role": "assistant",
                            "content": agent_result.content
                        }
                        finish_reason = agent_result.response_metadata["finish_reason"]
                        if (finish_reason):
                            message["content"] = ""

                        usage = {
                            "completion_tokens": agent_result.usage_metadata["output_tokens"],
                            "prompt_tokens": agent_result.usage_metadata["input_tokens"],
                            "total_tokens": agent_result.usage_metadata["total_tokens"]
                        }
                elif ("tools" in update):
                    tools = update["tools"]
                    tool_result = tools["messages"][0]
                    if (is_assistant):
                        message = {
                            "role": "assistant",
                            "step_details": {
                                "type": "tool_response",
                                "id": tool_result.id,
                                "tool_call_id": tool_result.tool_call_id,
                                "name": tool_result.name,
                                "content": tool_result.content
                            }
                        }
                    else:
                        message = {
                            "role": "tool",
                            "id": tool_result.id,
                            "tool_call_id": tool_result.tool_call_id,
                            "name": tool_result.name,
                            "content": tool_result.content
                        }
                else:
                    continue

            chunk_response = {
                "choices": [{
                    "index": 0,
                    "delta": message
                }]
            }
            if (finish_reason):
                chunk_response["choices"][0]["finish_reason"] = finish_reason
            if (usage):
                chunk_response["usage"] = usage
            yield chunk_response

    return generate, generate_stream


In [19]:
!pip install langchain_ibm

In [8]:
!pip install langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.5/152.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 12.5 MB/s eta 0:00:00


In [20]:
# Initialize AI Service function locally
from ibm_watsonx_ai.deployments import RuntimeContext

context = RuntimeContext(api_client=client)

streaming = False
findex = 1 if streaming else 0
local_function = gen_ai_service(context,  space_id=space_id)[findex]
messages = []

In [ ]:
local_question = "Summarize the latest research on sustainable energy."
messages.append({ "role" : "user", "content": local_question })
context = RuntimeContext(api_client=client, request_payload_json={"messages": messages})
response = local_function(context)
result = ''
if (streaming):
    for chunk in response:
        print(chunk, end="\n\n", flush=True)
else:
    print(response)


In [ ]:
# Look up software specification for the AI service
software_spec_id_in_project = "45f12dfe-aa78-5b8d-9f38-0ee223c47309"
software_spec_id = ""

try:
    software_spec_id = client.software_specifications.get_id_by_name("runtime-24.1-py3.11")
except:
    software_spec_id = client.spaces.promote(software_spec_id_in_project, source_project_id, space_id)

In [ ]:
# Define the request and response schemas for the AI service
request_schema = {
    "application/json": {
        "$schema": "http://json-schema.org/draft-07/schema#",
        "type": "object",
        "properties": {
            "messages": {
                "title": "The messages for this chat session.",
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "role": {
                            "title": "The role of the message author.",
                            "type": "string",
                            "enum": ["user","assistant"]
                        },
                        "content": {
                            "title": "The contents of the message.",
                            "type": "string"
                        }
                    },
                    "required": ["role","content"]
                }
            }
        },
        "required": ["messages"]
    }
}

response_schema = {
    "application/json": {
        "oneOf": [{"$schema":"http://json-schema.org/draft-07/schema#","type":"object","description":"AI Service response for /ai_service_stream","properties":{"choices":{"description":"A list of chat completion choices.","type":"array","items":{"type":"object","properties":{"index":{"type":"integer","title":"The index of this result."},"delta":{"description":"A message result.","type":"object","properties":{"content":{"description":"The contents of the message.","type":"string"},"role":{"description":"The role of the author of this message.","type":"string"}},"required":["role"]}}}}},"required":["choices"]},{"$schema":"http://json-schema.org/draft-07/schema#","type":"object","description":"AI Service response for /ai_service","properties":{"choices":{"description":"A list of chat completion choices","type":"array","items":{"type":"object","properties":{"index":{"type":"integer","description":"The index of this result."},"message":{"description":"A message result.","type":"object","properties":{"role":{"description":"The role of the author of this message.","type":"string"},"content":{"title":"Message content.","type":"string"}},"required":["role"]}}}}},"required":["choices"]}]
    }
}

In [ ]:
# Store the AI service in the repository
ai_service_metadata = {
    client.repository.AIServiceMetaNames.NAME: "Agent for research",
    client.repository.AIServiceMetaNames.DESCRIPTION: "",
    client.repository.AIServiceMetaNames.SOFTWARE_SPEC_ID: software_spec_id,
    client.repository.AIServiceMetaNames.CUSTOM: {},
    client.repository.AIServiceMetaNames.REQUEST_DOCUMENTATION: request_schema,
    client.repository.AIServiceMetaNames.RESPONSE_DOCUMENTATION: response_schema,
    client.repository.AIServiceMetaNames.TAGS: ["wx-agent"]
}

ai_service_details = client.repository.store_ai_service(meta_props=ai_service_metadata, ai_service=gen_ai_service)

In [ ]:
# Get the AI Service ID

ai_service_id = client.repository.get_ai_service_id(ai_service_details)

In [ ]:
# Deploy the stored AI Service
deployment_custom = {
    "avatar_icon": "DecisionTree",
    "avatar_color": "backgroundBrand",
    "placeholder_image": "placeholder2.png",
    "sample_questions": ["Summarize the latest research on sustainable energy.","Recent paper on the Agriculture development"]
}
deployment_metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Agent for research",
    client.deployments.ConfigurationMetaNames.ONLINE: {},
    client.deployments.ConfigurationMetaNames.CUSTOM: deployment_custom,
    client.deployments.ConfigurationMetaNames.DESCRIPTION: "AI research assistant agent for quick insights, summaries, and trend discovery.",
    client.repository.AIServiceMetaNames.TAGS: ["wx-agent"]
}

function_deployment_details = client.deployments.create(ai_service_id, meta_props=deployment_metadata, space_id=space_id)


## 4. Test AI Service

In [ ]:
# Get the ID of the AI Service deployment just created

deployment_id = client.deployments.get_id(function_deployment_details)
print(deployment_id)

01da35c1-d044-438b-9b1f-18306c72c530


In [ ]:
messages = []
remote_question = "Summarize the latest research on sustainable energy."
messages.append({ "role" : "user", "content": remote_question })
payload = { "messages": messages }

In [ ]:
result = client.deployments.run_ai_service(deployment_id, payload)
if "error" in result:
    print(result["error"])
else:
    print(result)

{'choices': [{'index': 0, 'message': {'content': 'The latest research on sustainable energy, from 2020 to the present, has seen significant advancements in various areas:\n\n1. **Smart Grids**: In 2022, a study simulated and analyzed "transactive energy mechanisms" to engage the large-scale deployment of flexible distributed energy resources (DERs), such as air conditioners, water heaters, batteries, and electric vehicles, in the operation of the electric power system.\n\n2. **Super Grids**: Researchers proposed a novel strategy in 2022 to create a global sustainable interconnected energy system based on deep-ocean-compressed hydrogen transportation.\n\n3. **Microgrids and Off-the-Grid Solutions**: A method for integrating multiple energy storage systems and distributed energy resources without dedicated communication or control systems was described in 2022. This approach could make microgrids more accessible and affordable, especially in areas where they are most needed, such as duri

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

def query_agent(question):
    """Sends a question to the deployed AI service and returns the response."""
    messages = [{"role": "user", "content": question}]
    payload = {"messages": messages}

    result = client.deployments.run_ai_service(deployment_id, payload)

    if "error" in result:
        return f"An error occurred: {result['error']}"
    elif "choices" in result and result["choices"]:
        return result["choices"][0]["message"]["content"]
    else:
        return "Sorry, I could not get a valid response from the service."

# Create and launch the Gradio interface
iface = gr.Interface(
    fn=query_agent,
    inputs=gr.Textbox(lines=5, label="Your Question",
    placeholder="e.g., Summarize the latest research on sustainable energy."),
    outputs=gr.Textbox(label="Agent's Response"),
    title="Agent for Research UI",
    description="Interact with the deployed AI research agent using this interface."
)

iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5fe00d107432e41f8d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
